In [3]:
using DataFrames, Feather

In [4]:
@enum FailureType begin
    # General ones
    Unknown
    InferredFailure
    PrintingChange
    TestAmbiguities
    NewAmbiguity
    ApproxError
    DocTest
    NewExceptionType
    DownloadError
    UnexpectedPass
    BadBoy
    SyntaxError
    VisualRegression
    MissingDep
    BuildError
    UnsatReq
    
    # Specific
    AtomicMacro
    InferenceStackOverFlow
    OldRevise
    DataTypeField
    BSONDataTypeField
    IllegalSparseBuffers
end


In [6]:
# Unpack the data unless it has already been unpacked

if !isdir("data")
    run(`tar -xvf data.tar.xz`)
end
primary  = Feather.read("data/primary.feather");
against = Feather.read("data/against.feather");

data/
data/primary.feather
data/against.json
data/against.feather
data/primary.json


In [8]:
# Join the primary and against data

package_results = leftjoin(primary, against,
     on=:uuid, makeunique=true, indicator=:source);

In [9]:
# These are packages that started failing when the RNG stream changed.
# These might have other problems but oh well...

rng_fails = 
["ACTRModels"
,"ADCME"
,"ARCHModels"
,"AbstractMCMC"
,"AdaptiveResonance"
,"Agents"
,"AllanDeviations"
,"Alpine"
,"ApproxBayes"
,"Arpack"
,"BandedMatrices"
,"BasicBSpline"
,"BasicInterpolators"
,"BayesNets"
,"BayesianExperiments"
,"CIAOAlgorithms"
,"CalculatedABC"
,"CalibrationTests"
,"Cassette"
,"CausalInference"
,"ClinicalTrialUtilities"
,"ConsistencyResampling"
,"Convex"
,"Convex1d"
,"CoordinateDescent"
,"Cropbox"
,"CrossEntropyMethod"
,"CumulantsFeatures"
,"DIVAnd"
,"DatagenCopulaBased"
,"Deconvolution"
,"DensityRatioEstimation"
,"DifferentialEvolutionMCMC"
,"DiscreteEvents"
,"DisjointCliqueCover"
,"Distributions"
,"DynamicHMC"
,"EMpht"
,"Equations"
,"Evolutionary"
,"ExactWrightFisher"
,"Extremes"
,"FINUFFT"
,"Faker"
,"Ferrite"
,"FeynmanKacParticleFilters"
,"ForwardDiff"
,"Fredholm"
,"FunctionWrappers"
,"GEEBRA"
,"GLM"
,"GSL"
,"GaussianFilters"
,"GaussianProcesses"
,"GeoEstimation"
,"GeoSimulation"
,"GeoStatsBase"
,"GeoTables"
,"HomotopyContinuation"
,"HypothesisTests"
,"IRKGaussLegendre"
,"ImageFeatures"
,"Infinity"
,"InterpolatedPDFs"
,"InvariantMeasures"
,"IterativeSolvers"
,"KLDivergences"
,"Kalman"
,"KissABC"
,"Knet"
,"LSHFunctions"
,"LorentzVectors"
,"LowLevelParticleFilters"
,"LowRankApprox"
,"LsqFit"
,"Luxor"
,"MCHammer"
,"MLDataPattern"
,"MLJScientificTypes"
,"MagnitudeDistributions"
,"MathOptSetDistances"
,"MatrixPencils"
,"MaximumLikelihoodProblems"
,"Meshes"
,"MinimalRLCore"
,"ModelingToolkit"
,"MonteCarloMeasurements"
,"NMF"
,"NaiveBayes"
,"NeXLMatrixCorrection"
,"Neighborhood"
,"NiLang"
,"Noise"
,"NonUniformRandomVariateGeneration"
,"NumericalAlgorithms"
,"OnlineStats"
,"OpenQuantumBase"
,"OpenQuantumSystems"
,"Optim"
,"PDSampler"
,"Perceptrons"
,"PermutationGroups"
,"Petri"
,"Photometry"
,"Plots"
,"PointProcessInference"
,"Poltergeist"
,"QuantumOpticsBase"
,"QuartetNetworkGoodnessFit"
,"RandomExtensions"
,"RecursiveFactorization"
,"ResettableStacks"
,"RiemannTheta"
,"RobotDynamics"
,"RobustAdaptiveMetropolisSampler"
,"SDDP"
,"ScatteredInterpolation"
,"ScoreDrivenModels"
,"SequentialSamplingModels"
,"SigmaRidgeRegression"
,"Simulate"
,"SpatialEcology"
,"StatsBase"
,"StochasticOptimalTransport"
,"StratiGraphics"
,"StressTest"
,"SymmetricTensors"
,"TabularMakie"
,"TexTables"
,"UncertaintyQuantification"
,"ValueOrientedRiskManagementInsurance"
,"Variography"
,"VoronoiCells"
,"VoronoiDelaunay"
,"WoodburyMatrices"
,"ZigZagBoomerang"
,""
,"Bukdu"
,"CorrelationFunctions"
,"FWFTables"
,"H3"
,"Jive"
,"LOLTools"
,"Millboard"
,"XUnit"
,""
,"DSP"
,"DataInterpolations"
,"NMRTools"
,"NearestNeighborDescent"
,"OptimalTransport"
,"Pitaya"
,"SCIP "
,"Tar "
,""
,"JetPackWaveFD"
,""
,""
,"AbstractOperators"
,"BayesianLinearRegression"
,"Controlz"
,"FilesystemDatastructures"
,"GAlgebra"
,"GeostatInversion"
,"IncompleteLU"
,"JuMP"
,"MCMCDiagnostics"
,"MixedModelsSim"
,"Nabla"
,"PLCTag"
,"PProf"
,"PSDMatrices"
,"PencilArrays"
,"PowerModels"
,"QuadEig"
,"RandomMatrices"
,"Reactive"
,"RemoveLFS"
,"Shapley"
,"SpatialJackknife"
,"TextClassification"] |> Set;

In [13]:
# Filter out packages that started failing but didn't start failing when the RNG stream changed

fails = filter(test->test.source == "both" &&
                 test.status != test.status_1 &&
                 !(test.name in rng_fails) &&
                 test.status in (":fail", ":kill"), package_results)

# Initially we do not know why a package failed
fails.why = fill(Unknown, size(fails, 1));

In [14]:
# Some utility functions

query(fails, s) = filter(row -> occursin(s, row[:log]), fails)    

function update_reason!(fails, needle, why)
    idxs = findall(row -> occursin(needle, row), fails.log)
    fails.why[idxs] .= why
    return fails
end

total_unknown(fails) = count(x -> x.why == Unknown, eachrow(fails))

total_unknown (generic function with 1 method)

In [25]:
# Here we pattern match certain test errors and categorize them based on that.

# Generic ones
update_reason!(fails, "Test.detect_ambiguities", TestAmbiguities)
update_reason!(fails, "detect_ambiguities(", TestAmbiguities)

update_reason!(fails, "Expression: all_doctests()", DocTest)
update_reason!(fails, "Error: doctest failure in ", DocTest)

update_reason!(fails, "does not match inferred return type", InferredFailure);
update_reason!(fails, "Expression: isapprox", ApproxError)
update_reason!(fails, r"Expression: (\S*) ≈ (\S*)", ApproxError) 
update_reason!(fails, "Expression: ≈(", ApproxError)
    
update_reason!(fails, "is ambiguous. Candidates:", NewAmbiguity)
update_reason!(fails, " ambiguities found", NewAmbiguity)

update_reason!(fails, "Unsatisfiable requirements detected for package", UnsatReq)



update_reason!(fails, "Evaluated: \"", PrintingChange)
update_reason!(fails, "Log Test Failed at", PrintingChange)
update_reason!(fails, "Expression: occursin(r\"", PrintingChange)
update_reason!(fails, "Expression: startswith(", PrintingChange)
update_reason!(fails, r"Expression: (\N.*?) == ", PrintingChange)
update_reason!(fails, "Evaluated: occursin(", PrintingChange)
update_reason!(fails, "Evaluated: endswith(", PrintingChange)
update_reason!(fails, "- DIFF ------------------------", PrintingChange)
update_reason!(fails, "LoadError: syntax", SyntaxError)

update_reason!(fails, "Image did not match reference image", VisualRegression)

update_reason!(fails, "      Thrown: ", NewExceptionType)
update_reason!(fails, "The requested URL returned error", DownloadError)
update_reason!(fails, "gzip: stdin: not in gzip format", DownloadError)
update_reason!(fails, "Unexpected Pass", UnexpectedPass)

update_reason!(fails, "Error building ", BuildError)



# Specific ones for this release
update_reason!(fails, "Illegal buffers for SparseMatrixCSC construction", IllegalSparseBuffers)


update_reason!(fails, "DataType has no field mutable", DataTypeField)
update_reason!(fails, "type DataType has no field ninitialized", DataTypeField)
update_reason!(fails, "@ BSON ~/.julia/packages/BSON/aEqHo/src/write.jl:46", BSONDataTypeField)




update_reason!(fails, "both CUDA and Base export \"@atomic\"", AtomicMacro)

update_reason!(fails, "Test.detect_ambiguities", TestAmbiguities)
update_reason!(fails, "intersect_aside", InferenceStackOverFlow);

update_reason!(fails, r"Package (\S.*) not found in current path:", MissingDep);



update_reason!(fails, "MethodError: no method matching Base.TOMLCache()", OldRevise)


nothing

In [26]:
# Packages that have gotten  

issues_opened = [
 "UnsteadyFlowSolvers",  # https://github.com/KiranUofG/UnsteadyFlowSolvers.jl/pull/46
 "Arpack", # https://github.com/JuliaLinearAlgebra/Arpack.jl/issues/132
 "RecursiveFactorization", #https://github.com/YingboMa/RecursiveFactorization.jl/issues/26
 "Phylo", # https://github.com/JuliaLang/julia/issues/41425
 "EmojiSymbols", # https://github.com/wookay/EmojiSymbols.jl/commit/35f97e0614d7d42cd7113263d7c841d5a659e841#commitcomment-52862922
]

likely_tol = [
];



In [27]:
# Total package failures that we haven't categorized

total_unknown(fails)

192

# Examples

Here are some examples of how one might do queries and categorize errors

In [29]:
# Unknown failures

filter(x -> x.why == Unknown && !(x.name in issues_opened), fails)

,julia,name,uuid
,String,String,String
1,"v""1.7.0-beta2-9a791aa09fb""",TBLIS,"UUID(""48530278-0828-4a49-9772-0f3830dfa1e9"")"
2,"v""1.7.0-beta2-9a791aa09fb""",Adapode,"UUID(""0750cfb5-909a-49d7-927e-29b6595444bf"")"
3,"v""1.7.0-beta2-9a791aa09fb""",ProgressMeter,"UUID(""92933f4c-e287-5a05-a399-4b506db050ca"")"
4,"v""1.7.0-beta2-9a791aa09fb""",AdaptiveDistanceFields,"UUID(""a1957575-6125-5dba-8f92-417d2d1f4a46"")"
5,"v""1.7.0-beta2-9a791aa09fb""",Continuables,"UUID(""79afa230-ca09-11e8-120b-5decf7bf5e25"")"
6,"v""1.7.0-beta2-9a791aa09fb""",SortingAlgorithms,"UUID(""a2af1166-a08f-5f64-846c-94a0d3cef48c"")"
7,"v""1.7.0-beta2-9a791aa09fb""",DECAES,"UUID(""d84fb938-a666-558e-89d9-d531edc6724f"")"
8,"v""1.7.0-beta2-9a791aa09fb""",Remark,"UUID(""79b45036-8e38-5d04-8f49-b9fb23ff5a0d"")"
9,"v""1.7.0-beta2-9a791aa09fb""",MLJFlux,"UUID(""094fc8d1-fd35-5302-93ea-dabda2abf845"")"


In [30]:
# Categorize

z = []
for i in instances(FailureType)
    n = count(row -> row.why == i, eachrow(fails))
    n > 0 && push!(z, (n, i))
end
sort!(z; rev=true)

17-element Vector{Any}:
 (192, Unknown)
 (46, PrintingChange)
 (45, DataTypeField)
 (22, ApproxError)
 (15, NewAmbiguity)
 (12, BSONDataTypeField)
 (11, IllegalSparseBuffers)
 (11, DocTest)
 (10, NewExceptionType)
 (5, AtomicMacro)
 (5, UnsatReq)
 (5, SyntaxError)
 (4, InferenceStackOverFlow)
 (4, InferredFailure)
 (2, BuildError)
 (2, TestAmbiguities)
 (1, VisualRegression)

In [32]:
# Package names of some particular failure

d = filter(x -> x.why == BSONDataTypeField, fails).name
d = filter(!in(issues_opened), d)
d = filter(!in(likely_tol), d)
join(d, '\n') |> print

Checkpoints
BSON
CachedCalls
MLJSerialization
BrokenRecord
FHIRClientXML
Swalbe
FluxTraining
Discord
PubChemCrawler
FilePathsBase
DeepQLearning